In [ ]:
from ipywidgets import widgets, interact
import os

In [ ]:
user = 'mark'

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var command = ["currenturl = ", "'", window.location.href, "'" ].join('')
kernel.execute(command)

In [ ]:
from urllib.parse import urlparse
o = urlparse(currenturl)
hostname = o.hostname

In [ ]:
from IPython.display import display, HTML
url = "<iframe src=http://" + hostname + ":8080/?user=" + user + " width=700 height=500></iframe>"
HTML(url)

In [ ]:
modelfile = os.path.join('model', user + '.py')

In [ ]:
%load $modelfile